In [1]:
import sys 
sys.path.append("..")
import os
import numpy as np
import pandas as pd
data_dir="./sz50/"

In [2]:
model_list=os.listdir("/home/sunshuo/qml/RL_Mix/PM/data/portfolio_management/sunrise-pm/sz50")
# the all_file_list are the models we need
#here we use sr as our test score and do the trick
test_sr_average=[]# this is the list to store every models' average result 
for i in range(len(model_list)):
    result_sr=[]# this is the list to store single models result  
    model_path=os.path.join(data_dir,model_list[i])# find one model 
    seed_file_list=os.listdir(model_path)# find its seed 
    for seed in seed_file_list:
        result=pd.read_csv( os.path.join(data_dir,model_list[i],seed,"result.csv"),index_col=0)
        num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"])) # find the num_epoch in that random seed to stop and see
        seed_best_result=result.iloc[num_epoch]["test_tr"]
        result_sr.append(seed_best_result)
    average_result=np.average(result_sr)
    test_sr_average.append(average_result)
    # print(test_sr_average,model_list[i] )

loc=test_sr_average.index(max(test_sr_average))
print("the best model is ",model_list[loc])
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
best_results=[]
best_crs=[]
returns_list=[]
for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[loc],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    print("the",seed,"'s best train result in the valid is",num_epoch)
    returns=pd.read_csv( os.path.join(data_dir,model_list[loc],seed,"test_daily_return_{}.csv".format(num_epoch)),index_col=0)
    returns=np.array(returns["daily_return"]).tolist()
    returns_list.append(returns)
    seed_best_result=result.iloc[num_epoch]["test_sr"]
    seed_best_cr=result.iloc[num_epoch]["test_tr"]
    best_results.append(seed_best_result)
    best_crs.append(seed_best_cr)
average_best_result=np.mean(best_results)
average_cr=np.mean(best_crs)
print("the best sr result is ",average_best_result)   
print("the best tr result is ",average_cr*100,"%")   
np.save("Mixsz50.npy",np.array(returns_list)) 

the best model is  en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.5
the seed_51234 's best train result in the valid is 7
the seed_45123 's best train result in the valid is 4
the seed_34512 's best train result in the valid is 2
the seed_23451 's best train result in the valid is 2
the seed_12345 's best train result in the valid is 8
the best sr result is  0.42422131268289043
the best tr result is  13.764689920784907 %


the best baseline is from sac and  cr:377.76±982.90    sr:3.03±0.87

In [3]:
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
results=[]
for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[loc],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    result=pd.read_csv(os.path.join(data_dir,model_list[loc],seed,"test_daily_return_{}.csv".format(num_epoch)),index_col=0)
    results.append(result)
for result in results:
    result["reward"]=result.daily_return+1
    y_list=[1]
    for x in list(result.reward):
        y=y_list[-1]*x
        y_list.append(y)
    y_list=y_list[1:]
    result["Cumlative Return"]=y_list
result_1,result_2,result_3,result_4,result_5=results
average_df=pd.DataFrame()
average_df["daily_return"]=(result_1["daily_return"]+result_2["daily_return"]+result_3["daily_return"]+result_4["daily_return"]+result_5["daily_return"])/5
average_df["reward"]=average_df["daily_return"]+1
y_list=[1]
for x in list(average_df.reward):
    y=y_list[-1]*x
    y_list.append(y)
y_list=y_list[1:]
average_df["Cumlative Return"]=y_list
average_df["result1"]=result_1["Cumlative Return"]
average_df["result2"]=result_2["Cumlative Return"]
average_df["result3"]=result_3["Cumlative Return"]
average_df["result4"]=result_4["Cumlative Return"]
average_df["result5"]=result_5["Cumlative Return"]
average_df["std"]=np.std([average_df.result1,average_df.result2,average_df.result3,average_df.result4,average_df.result5],axis=0)
average_df.to_csv("/home/sunshuo/qml/RL_Mix/sz50exen.csv")

In [4]:
def evaualte(df):
    daily_return=df["daily_return"]
    neg_ret_lst=df[df["daily_return"]<0]["daily_return"]
    sharpe_ratio=np.mean(daily_return)/np.std(daily_return)*(len(df)**0.5)
    vol=np.std(daily_return)
    mdd=max((max(df["Cumlative Return"])-df["Cumlative Return"])/max(df["Cumlative Return"]))
    cr=np.sum(daily_return)/mdd
    sor=np.sum(daily_return)/np.std(neg_ret_lst)/np.sqrt(len(daily_return))
    return sharpe_ratio,vol,mdd,cr,sor
def all(results):
    sharpe_ratios=[]
    vols=[]
    mdds=[]
    crs=[]
    sors=[]
    for result in results:
        sr,vo,mdd,cr,sor=evaualte(result)
        sharpe_ratios.append(sr)
        vols.append(vo)
        mdds.append(mdd)
        crs.append(cr)
        sors.append(sor)


        
        
    print("sr_average:{},sr_std:{}".format(np.mean(sharpe_ratios),np.std(sharpe_ratios)))
    print("cr_average:{},cr_std:{}".format(np.mean(crs),np.std(crs)))
    print("sor_average:{},sor_std:{}".format(np.mean(sors),np.std(sors) ))
    print("mdd_average:{},mdd_std:{}".format(np.mean(mdds),np.std(mdds) ))
    print("vol_average:{},vol_std:{}".format(np.mean(vols),np.std(vols) ))
all(results)

sr_average:0.7563256171452735,sr_std:0.12137778908342486
cr_average:0.5212244753607925,cr_std:0.06190739537013845
sor_average:0.8349855284715122,sor_std:0.13545166211405132
mdd_average:0.2817396289250719,mdd_std:0.019066738343010252
vol_average:0.006897265715318709,vol_std:0.00021098673690743555


In [5]:
import sys
sys.path.append("../../..")
from rl_env.portfolio_management import get_dataset_config
train_config, valid_config, test_config=get_dataset_config("sz50")
test=test_config["df"]
def sl_targetmaking(df):
    """making target, aka the future profit margin"""
    df_new=df.sort_values(by=["tic", "date"])
    unique_ticker = df_new.tic.unique()
    df_indicator=pd.DataFrame()
    for i in range(len(unique_ticker)):
        temp_indicator = df_new[df_new.tic == unique_ticker[i]]
        target=temp_indicator.zclose[1:]
        target=np.append(target,np.nan)
        temp_indicator["target"]=target
        df_indicator=df_indicator.append(temp_indicator,ignore_index=True)
        
    df_indicator = df_indicator.fillna(method="ffill").fillna(method="bfill")
    df_indicator.index = df_indicator.date.factorize()[0]
    return df_indicator
new_test=sl_targetmaking(test)
def softmax_normalization(actions):
    numerator = np.exp(actions)
    denominator = np.sum(np.exp(actions))
    softmax_output = numerator/denominator
    return softmax_output
def log(action):
    action=np.log(action)
    return action
def get_kde(x,data_array,bandwidth):
    def gauss(x):
        import math
        return (1/math.sqrt(2*math.pi))*math.exp(-0.5*(x**2))
    N=len(data_array)
    res=0
    if len(data_array)==0:
        return 0
    for i in range(len(data_array)):
        res += gauss((x-data_array[i])/bandwidth)
    res /= (N*bandwidth)
    return res
def lhkdekde(x,data_array,bandwidth):
    return(np.log(get_kde(x,data_array=data_array,bandwidth=bandwidth))*get_kde(x,data_array=data_array,bandwidth=bandwidth))
def get_H(x,data_array,bandwidth):
    # 需要计算一下下限
    low=min(data_array)
    std=np.std(data_array)
    low=low-2*std
    from scipy import integrate
    def f(x):
        return(lhkdekde(x,data_array=data_array,bandwidth=bandwidth))
    return integrate.quad(f,a=low,b=x)
def get_entropy(actions):
    entropys=[]
    for action in actions:
        entropy=-action.dot(log(action))
        entropys.append(entropy)
    entropy_mean=np.mean(entropys)
    return entropy_mean
def get_ENB(actions):
    ENBs=[]
    for action in actions:
        w=action[1:]
        w_f=np.array(np.linalg.inv(eigVects).dot(w))
        w_f=w_f[0]
        w_f2=w_f*w_f
        eigVals2=eigVals*eigVals
        t=eigVals2*w_f2
        pw=t/np.sum(t)
        ENB=np.exp(-np.sum(pw*np.log(pw)))
        ENBs.append(ENB)
    return np.mean(ENBs)
new_test=new_test[["date","tic","target"]]
new_test = new_test.sort_values(["date", "tic"], ignore_index=True)
new_dataframe=pd.DataFrame()
for date in new_test.date.unique():
    a=new_test[new_test.date==date]
    b=pd.DataFrame(data=[list(a.target)],columns=list(a.tic),index=a.date.unique())
    new_dataframe=new_dataframe.append(b)
covMat=np.cov(new_dataframe,rowvar=0)
eigVals,eigVects=np.linalg.eig(np.mat(covMat))
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
entropy_means=[]
ENB_mean=[]
for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[loc],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    actions=np.load(os.path.join(data_dir,model_list[loc],seed,"test_daily_action_{}.npy".format(num_epoch)))
    new_action=[]
    for action in actions:
        action=softmax_normalization(action).squeeze()
        new_action.append(action)
    actions=new_action
    entropy_means.append(get_entropy(actions))
    ENB_mean.append(get_ENB(actions))

print("entropy mean",np.mean(entropy_means))
print("entropy std",np.std(entropy_means))

print("ENB mean",np.mean(ENB_mean))
print("ENB std",np.std(ENB_mean))
np.save("/home/sunshuo/qml/RL_Mix/diversity/sz50/Entropy.npy",entropy_means)

../../../rl_env/portfolio_management.py:303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_indicator["zclose"]=(temp_indicator.close/(temp_indicator.close.rolling(2).sum()-temp_indicator.close))-1
../../../rl_env/portfolio_management.py:304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_indicator["zd_5"]=(temp_indicator.adjcp.rolling(5).sum()/5)/temp_indicator.adjcp-1
../../../rl_env/portfolio_management.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

entropy mean 3.1273227
entropy std 0.027535968
ENB mean 1.025773809044568
ENB std 0.008091870664550267


In [6]:
test_sr_average=[]# this is the list to store every models' average result 
for i in range(len(model_list)):
    result_sr=[]# this is the list to store single models result  
    model_path=os.path.join(data_dir,model_list[i])
    seed_file_list=os.listdir(model_path)
    for seed in seed_file_list:
        result=pd.read_csv( os.path.join(data_dir,model_list[i],seed,"result.csv"),index_col=0)
        num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"])) # find the num_epoch in that random seed to stop and see
        seed_best_result=result.iloc[num_epoch]["test_tr"]
        result_sr.append(seed_best_result)
    average_result=np.average(result_sr)
    test_sr_average.append(average_result)
    # print(test_sr_average,model_list[i] )
loc=test_sr_average.index(max(test_sr_average))
print("the best model is ",model_list[loc])
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
best_results=[]
best_crs=[]

for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[i],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    print("the",seed,"'s best train result in the valid is",num_epoch)
    seed_best_result=result.iloc[num_epoch]["test_sr"]
    seed_best_cr=result.iloc[num_epoch]["test_tr"]
    best_results.append(seed_best_result)
    best_crs.append(seed_best_cr)
average_best_result=np.mean(best_results)
average_cr=np.mean(best_crs)
print("the best sr result is ",average_best_result)   
print("the best tr result is ",average_cr*100,"%")   
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
results=[]
for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[i],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    result=pd.read_csv(os.path.join(data_dir,model_list[i],seed,"test_daily_return_{}.csv".format(num_epoch)),index_col=0)
    results.append(result)
for result in results:
    result["reward"]=result.daily_return+1
    y_list=[1]
    for x in list(result.reward):
        y=y_list[-1]*x
        y_list.append(y)
    y_list=y_list[1:]
    result["Cumlative Return"]=y_list
result_1,result_2,result_3,result_4,result_5=results
average_df=pd.DataFrame()
average_df["daily_return"]=(result_1["daily_return"]+result_2["daily_return"]+result_3["daily_return"]+result_4["daily_return"]+result_5["daily_return"])/5
average_df["reward"]=average_df["daily_return"]+1
y_list=[1]
for x in list(average_df.reward):
    y=y_list[-1]*x
    y_list.append(y)
y_list=y_list[1:]
average_df["Cumlative Return"]=y_list
average_df["result1"]=result_1["Cumlative Return"]
average_df["result2"]=result_2["Cumlative Return"]
average_df["result3"]=result_3["Cumlative Return"]
average_df["result4"]=result_4["Cumlative Return"]
average_df["result5"]=result_5["Cumlative Return"]
average_df["std"]=np.std([average_df.result1,average_df.result2,average_df.result3,average_df.result4,average_df.result5],axis=0)
average_df


the best model is  en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.5
the seed_51234 's best train result in the valid is 7
the seed_45123 's best train result in the valid is 4
the seed_34512 's best train result in the valid is 2
the seed_23451 's best train result in the valid is 2
the seed_12345 's best train result in the valid is 8
the best sr result is  0.42422131268289043
the best tr result is  13.764689920784907 %


,daily_return,reward,Cumlative Return,result1,result2,result3,result4,result5,std
0,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.008170,1.008170,1.008170,1.009285,1.008802,1.007459,1.008806,1.006498,0.001034
2,-0.003065,0.996935,1.005080,1.006363,1.005336,1.003743,1.005630,1.004325,0.000935
3,0.001696,1.001696,1.006784,1.008399,1.007343,1.005688,1.007897,1.004593,0.001426
4,-0.002524,0.997476,1.004243,1.006024,1.005024,1.002651,1.005779,1.001740,0.001729
...,...,...,...,...,...,...,...,...,...
795,-0.005581,0.994419,1.109480,1.117055,1.111268,1.115483,1.060091,1.143487,0.027175
796,0.006528,1.006528,1.116722,1.124332,1.119755,1.122888,1.066726,1.149878,0.027218
797,0.005311,1.005311,1.122654,1.129846,1.126764,1.129186,1.071465,1.156021,0.027731
798,0.007817,1.007817,1.131429,1.139113,1.136257,1.138279,1.079605,1.163890,0.027816


In [7]:
def evaualte(df):
    daily_return=df["daily_return"]
    neg_ret_lst=df[df["daily_return"]<0]["daily_return"]
    sharpe_ratio=np.mean(daily_return)/np.std(daily_return)*(len(df)**0.5)
    vol=np.std(daily_return)
    mdd=max((max(df["Cumlative Return"])-df["Cumlative Return"])/max(df["Cumlative Return"]))
    cr=np.sum(daily_return)/mdd
    sor=np.sum(daily_return)/np.std(neg_ret_lst)/np.sqrt(len(daily_return))
    return sharpe_ratio,vol,mdd,cr,sor
def all(results):
    sharpe_ratios=[]
    vols=[]
    mdds=[]
    crs=[]
    sors=[]
    for result in results:
        sr,vo,mdd,cr,sor=evaualte(result)
        sharpe_ratios.append(sr)
        vols.append(vo)
        mdds.append(mdd)
        crs.append(cr)
        sors.append(sor)


        
        
    print("sr_average:{},sr_std:{}".format(np.mean(sharpe_ratios),np.std(sharpe_ratios)))
    print("cr_average:{},cr_std:{}".format(np.mean(crs),np.std(crs)))
    print("sor_average:{},sor_std:{}".format(np.mean(sors),np.std(sors) ))
    print("mdd_average:{},mdd_std:{}".format(np.mean(mdds),np.std(mdds) ))
    print("vol_average:{},vol_std:{}".format(np.mean(vols),np.std(vols) ))
all(results)

sr_average:0.7563256171452735,sr_std:0.12137778908342486
cr_average:0.5212244753607925,cr_std:0.06190739537013845
sor_average:0.8349855284715122,sor_std:0.13545166211405132
mdd_average:0.2817396289250719,mdd_std:0.019066738343010252
vol_average:0.006897265715318709,vol_std:0.00021098673690743555


In [8]:
test_sr_average=[]# this is the list to store every models' average result 
model_list=['en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.9',
 'en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.7','en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.3',
  'en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.1','en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20']
for i in range(len(model_list)):
    result_sr=[]# this is the list to store single models result  
    model_path=os.path.join(data_dir,model_list[i])
    seed_file_list=os.listdir(model_path)
    for seed in seed_file_list:
        result=pd.read_csv( os.path.join(data_dir,model_list[i],seed,"result.csv"),index_col=0)
        num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"])) # find the num_epoch in that random seed to stop and see
        seed_best_result=result.iloc[num_epoch]["test_sr"]
        result_sr.append(seed_best_result)
    average_result=np.average(result_sr)
    test_sr_average.append(average_result)
    # print(test_sr_average,model_list[i] )
    print(model_list[i])
    print(average_result)
loc=test_sr_average.index(max(test_sr_average))
print(max(test_sr_average))
print("the best model is ",model_list[loc])

FileNotFoundError: [Errno 2] No such file or directory: './sz50/en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_10000_discount_0.99_tem_20_bm_0.5_uncertain_0.9'

In [ ]:
model_list=os.listdir("/home/sunshuo/qml/RL_Mix/PM/data/portfolio_management/sunrise-pm/crypto")
# the all_file_list are the models we need

In [ ]:
test_sr_average=[]# this is the list to store every models' average result 
for i in range(len(model_list)):
    results=[]
    result_sr=[]# this is the list to store single models result  
    model_path=os.path.join(data_dir,model_list[i])
    seed_file_list=os.listdir(model_path)
    for seed in seed_file_list:
        result=pd.read_csv( os.path.join(data_dir,model_list[i],seed,"result.csv"),index_col=0)
        num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"])) # find the num_epoch in that random seed to stop and see
        result=pd.read_csv(os.path.join(data_dir,model_list[i],seed,"test_daily_return_{}.csv".format(num_epoch)),index_col=0)
        results.append(result)
    for result in results:
        # print(result)
        result["reward"]=result.daily_return+1
        # print(result)
        y_list=[1]
        for x in list(result.reward):
            y=y_list[-1]*x
            y_list.append(y)
        y_list=y_list[1:]
        result["Cumlative Return"]=y_list
    result_1,result_2,result_3,result_4,result_5=results
    average_df=pd.DataFrame()
    average_df["daily_return"]=(result_1["daily_return"]+result_2["daily_return"]+result_3["daily_return"]+result_4["daily_return"]+result_5["daily_return"])/5
    average_df["reward"]=average_df["daily_return"]+1
    y_list=[1]
    for x in list(average_df.reward):
        y=y_list[-1]*x
        y_list.append(y)
    y_list=y_list[1:]
    average_df["Cumlative Return"]=y_list
    average_result=y_list[-1]
    test_sr_average.append(average_result)
    # print(test_sr_average,model_list[i] )

loc=test_sr_average.index(max(test_sr_average))
print("the best model is ",model_list[loc])
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
best_results=[]
best_crs=[]

for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[loc],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    print("the",seed,"'s best train result in the valid is",num_epoch)
    seed_best_result=result.iloc[num_epoch]["test_sr"]
    seed_best_cr=result.iloc[num_epoch]["test_tr"]
    best_results.append(seed_best_result)
    best_crs.append(seed_best_cr)
average_best_result=np.mean(best_results)
average_cr=np.mean(best_crs)
print("the best sr result is ",average_best_result)   
print("the best tr result is ",average_cr*100,"%")   

the best model is  en_3_batch_256_plr_0.0007_qlr_0.0007_layer_128_2_buffer_1000_discount_0.99_tem_20_bm_0.5_uncertain_0.9
the seed_51234 's best train result in the valid is 6
the seed_45123 's best train result in the valid is 9
the seed_34512 's best train result in the valid is 6
the seed_23451 's best train result in the valid is 5
the seed_12345 's best train result in the valid is 2
the best sr result is  2.110420553303385
the best tr result is  293.33554422749046 %


In [ ]:
model_path=os.path.join(data_dir,model_list[loc])
seed_file_list=os.listdir(model_path)
results=[]
for seed in seed_file_list:
    result=pd.read_csv( os.path.join(data_dir,model_list[loc],seed,"result.csv"),index_col=0)
    num_epoch=list(result["eval_sr"]).index(max(result["eval_sr"]))
    result=pd.read_csv(os.path.join(data_dir,model_list[i],seed,"test_daily_return_{}.csv".format(num_epoch)),index_col=0)
    results.append(result)
for result in results:
    result["reward"]=result.daily_return+1
    y_list=[1]
    for x in list(result.reward):
        y=y_list[-1]*x
        y_list.append(y)
    y_list=y_list[1:]
    result["Cumlative Return"]=y_list
result_1,result_2,result_3,result_4,result_5=results
average_df=pd.DataFrame()
average_df["daily_return"]=(result_1["daily_return"]+result_2["daily_return"]+result_3["daily_return"]+result_4["daily_return"]+result_5["daily_return"])/5
average_df["reward"]=average_df["daily_return"]+1
y_list=[1]
for x in list(average_df.reward):
    y=y_list[-1]*x
    y_list.append(y)
y_list=y_list[1:]
average_df["Cumlative Return"]=y_list
average_df["result1"]=result_1["Cumlative Return"]
average_df["result2"]=result_2["Cumlative Return"]
average_df["result3"]=result_3["Cumlative Return"]
average_df["result4"]=result_4["Cumlative Return"]
average_df["result5"]=result_5["Cumlative Return"]
average_df["std"]=np.std([average_df.result1,average_df.result2,average_df.result3,average_df.result4,average_df.result5],axis=0)
average_df

,daily_return,reward,Cumlative Return,result1,result2,result3,result4,result5,std
0,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.148552,1.148552,1.148552,1.162694,1.154111,1.150226,1.104844,1.170887,0.022991
2,0.042918,1.042918,1.197846,1.198063,1.212618,1.187775,1.172540,1.217205,0.016338
3,0.020475,1.020475,1.222372,1.208272,1.243575,1.201954,1.223155,1.233296,0.015393
4,0.032605,1.032605,1.262227,1.237562,1.276436,1.250622,1.271870,1.272811,0.015161
...,...,...,...,...,...,...,...,...,...
181,-0.039723,0.960277,4.183953,4.562124,5.830302,2.730863,3.787989,4.255202,1.011399
182,0.003584,1.003584,4.198948,4.590343,5.834572,2.737280,3.801880,4.276411,1.011584
183,0.013805,1.013805,4.256915,4.654111,5.901264,2.772559,3.869690,4.331920,1.020640
184,0.016770,1.016770,4.328304,4.721665,5.982814,2.821187,3.961742,4.393386,1.028658
